In [1]:
# Setup the notebook based on running environment.
import os
# Optional: Enable telemetry in browser_use and chromadb
os.environ["ANONYMIZED_TELEMETRY"] = "false"
try:
    from kaggle_secrets import UserSecretsClient # type: ignore
except Exception as e:
    class UserSecretsClient:
        @classmethod
        def set_secret(cls, id: str, value: str):
            os.environ[id] = value
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU google-genai==1.45.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core lmnr[all] browser-use
    from browser_use import Agent as BrowserAgent
else:
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-ai-generativelanguage pydrive2 tensorflow cryptography pyOpenSSL langchain langchain-core nltk ray click google-generativeai google-cloud-translate datasets cesium bigframes plotnine mlxtend fastai spacy thinc google-colab gcsfs jupyter-kernel-gateway
    !pip install -qU google-genai==1.45.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    !pip install -qU langchain-community langchain-text-splitters wikipedia lmnr[all]

import ast, chromadb, json, logging, pandas, platform, pytz, re, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from enum import Enum
from google import genai
from google.api_core import retry, exceptions
from google.genai import types
from IPython.display import Markdown, display
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from lmnr import Laminar
from math import inf
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable, NewType
from wikipedia.exceptions import DisambiguationError, PageError

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 38.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107

In [2]:
# Prepare the Gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Activate Laminar auto-instrumentation.
try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

class GeminiModel:
    def __init__(self, rpm: list, tpm: list, rpd: list):
        self.rpm = rpm # requests per minute
        self.tpm = tpm # tokens per minute in millions
        self.rpd = rpd # requests per day
        self.err = [0,0] # validation, api_related

# A python api-helper with model fail-over/chaining/retry support.
GeminiEmbedFunction = NewType("GeminiEmbedFunction", None) # forward-decl
class Gemini:
    gen_limit_in = 1048576
    emb_limit_in = 2048
    gen_model = {
        "gemini-2.0-flash": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # stable wo/thinking: 15 RPM/1M TPM/200 RPD
        "gemini-2.0-flash-exp": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # latest w/thinking: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash-preview-09-2025": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # exp: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-flash": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # stable: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-flash-lite-preview-09-2025": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # exp: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-flash-lite": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # stable: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-pro": GeminiModel([5,150,1000,2000],[.25,2,5,8],[100,10000,50000,inf]), # stable: 5 RPM/250K TPM/100 RPD
    }
    gen_local = []
    default_model = []
    embed_model = "gemini-embedding-001", GeminiModel([100,3000,5000,10000],[.03,1,5,10],[1000,inf,inf,inf]) # stable: 100 RPM/30K TPM/1000 RPD/100 per batch
    error_total = 0
    min_rpm = 3
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 45.0
    dt_rpm = 60.0

    @classmethod
    def get(cls, url: str):
        # Create a header matching the OS' tcp-stack fingerprint.
        system_ua = None
        match platform.system():
            case 'Linux':
                system_ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
            case 'Darwin':
                system_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 15_7_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0 Safari/605.1.15'
            case 'Windows':
                system_ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
        try:
            request = requests.get(url, headers={'User-Agent': system_ua})
            if request.status_code != requests.codes.ok:
                print(f"api.get() returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    class Limit(Enum):
        FREE = 0
        TIER_1 = 1
        TIER_2 = 2
        TIER_3 = 3
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value

    def __init__(self, with_limit: Limit, default_model: str):
        self.client = genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY"))
        self.limit = with_limit.value
        self.m_id = list(self.gen_model.keys()).index(default_model)
        self.default_model.append(default_model)
        self.default_local = default_model
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        self.s_embed = GeminiEmbedFunction(self.client, semantic_mode = True)
        logging.getLogger("google_genai").setLevel(logging.WARNING) # suppress info on generate

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + list(self.gen_model.keys())[self.m_id]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0]

    def push_default_model(self, model_code: str):
        if model_code in self.gen_model.keys():
            self.stop_running()
            self.default_model.append(model_code)
            self.m_id = list(self.gen_model.keys()).index(model_code)
        else:
            print(f"{model_code} not found in gen_model.keys()")

    def pop_default_model(self):
        self.stop_running()
        self.default_model.pop(-1)
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])

    def set_default_local(self, model_index: int):
        if model_index in range(0, len(self.gen_local)):
            self.default_local = model_index
        else:
            print(f"set default local({model_index}) must be 0..{len(self.gen_local)-1}")

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(len(self.gen_model.keys())):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except (genai.errors.APIError, exceptions.RetryError) as api_error:
                if isinstance(api_error, genai.errors.APIError):
                    retriable = api_error.code in {429, 503, 500, 400} # code 400 when TPM exceeded
                    if not retriable or attempt == len(self.gen_model.keys())-1:
                        raise api_error
                self.on_error(kwargs)
            except Exception as e:
                raise e

    def on_error(self, kwargs):
        self.generation_fail()
        kwargs["model"] = self(Gemini.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        list(self.gen_model.values())[self.m_id].err[0] += 1
        self.error_total += 1

    def generation_fail(self):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("api.generation_fail.next_model: model is now ", list(self.gen_model.keys())[self.m_id])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True

    def save_error(self):
        list(self.gen_model.values())[self.m_id].err[1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model.keys())
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]

    def refill_rpm(self):
        self.running = False
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        print("api.refill_rpm ", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        print("api.zero_error: model is now ", list(self.gen_model.keys())[self.m_id])

    def token_count(self, expr: str):
        count = self.client.models.count_tokens(
            model=self(Gemini.Model.GEN),
            contents=json.dumps(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for m_code, m in self.gen_model.items():
            errors["by_model"].update({
                m_code: {
                    "api_related": m.err[1],
                    "validation": m.err[0]
                }})
        return errors

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, content: list):
        return self.s_embed.sts(content)

Skipping Laminar.initialize()


In [3]:
# Define the embedding function.
api = NewType("Gemini", None) # forward-decl
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Gemini.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Gemini.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [4]:
# Instantiate the api-helper with usage limit.
api = Gemini(with_limit=Gemini.Limit.FREE, default_model="gemini-2.0-flash")

# Gemini Baseline Checks

In [5]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   A stock market is a place (or system) where stocks, bonds, and other securities are bought and sold. These exchanges can be physical locations or virtual platforms.
*   It allows companies to raise capital by selling shares of ownership to the public.
*   Investors purchase these shares hoping to profit as the company grows and the value of the stock increases.

**How it works:**

*   Stockbrokers and traders buy and sell shares on behalf of their clients or themselves.
*   The price of a stock is determined by supply and demand. If more people want to buy a stock than sell it, the price goes up, and vice versa.
*   Many stocks are traded on exchanges, while others are traded "over the counter" (OTC) through dealers.

**Importance:**

*   The stock market is considered a primary indicator of a country's economic strength and development.
*   Rising stock prices are often associated with increased business investment and economic growth.
*   Stock prices also affect household wealth and consumer spending.

**Key Concepts:**

*   **Market capitalization:** The total value of a company's outstanding shares.
*   **Stock Exchange:** A place where stocks are bought and sold. Examples include the New York Stock Exchange (NYSE) and the Nasdaq.
*   **Indices:** Measures of the performance of a group of stocks, such as the Dow Jones Industrial Average (DJI), S&P 500, and Nasdaq Composite.

**Current Market Conditions (as of November 7, 2025):**

*   US stock markets are experiencing some volatility.
*   The Dow Jones Industrial Average is down, as is the S&P 500 and the Nasdaq.
*   Concerns about the US economy, high tech valuations, and the potential bursting of an AI bubble are contributing to the market jitters.

Please note that this information is for general knowledge only and should not be considered financial advice.


In [6]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Here's an overview of what analysts are saying about AMZN (Amazon) stock as of November 2025:

**Current Price and Market Sentiment:**

*   As of November 7, 2025, AMZN is trading around $243.44.
*   The stock has fluctuated between $238.49 and $245.50 recently.
*   Over the past year, AMZN has traded between $161.38 and $258.60.
*   Overall, analysts have a "Strong Buy" or "Moderate Buy" consensus rating on the stock.

**Analyst Price Targets:**

*   The average 12-month price target from analysts is around $293.17 to $296.26.
*   Some analysts have a high forecast of $340.00.
*   The median price target is $291.95, suggesting a potential upside of over 20%.

**Forecasts for 2025:**

*   Predictions vary, but one source suggests a trading channel between $231.05 and $255.56 in 2025, with an average price of $239.63.
*   Another projection estimates the share price could fluctuate between $242.89 and $296.87 by the end of 2025, averaging near $269.88.
*   One source predicts a year-end price of $242.71.

**Factors Influencing the Stock:**

*   **Earnings:** Amazon's Q3 earnings beat expectations, driven by AI-fueled AWS growth.
*   **Revenue Growth:** Analysts expect revenue to grow around 10% in 2025.
*   **AWS:** Amazon Web Services is a significant driver of revenue and operating income.
*   **AI:** Amazon's partnership with OpenAI and focus on AI are seen as positives.
*   **Valuation:** Some analysts believe the stock's valuation is attractive.
*   **Competition:** Amazon is facing competition in e-commerce from companies like Temu.
*   **Regulation:** Regulatory concerns remain a risk for large tech companies like Amazon.

**Long-Term Projections:**

*   One model projects an average price of around $879.66 by mid-2030 and approximately $969.89 by the end of 2030.
*   In 2035, the stock could average around $1,479.21.


In [7]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's a summary of the current share price, short-term trends, and bullish/bearish predictions for AMZN (Amazon) stock as of November 8, 2025:

**Current Share Price:**

*   The current price of AMZN is around $244.41.
*   It has increased by about 0.56% in the last 24 hours.
*   The stock has fluctuated between $238.49 and $245.50 recently.

**Short-Term Trends:**

*   **Mixed Signals:** AMZN stock has fallen by -2.28% compared to the previous week, but has risen 9.64% over the past month. Over the last year, AMZN has shown a 17.82% increase.
*   **Horizontal Trend:** The stock is moving within a wide and horizontal trend, and further movements within this trend can be expected.
*   **Positive Signals:** AMZN holds buy signals from both short and long-term Moving Averages, giving a positive forecast for the stock. There is also a buy signal from the 3-month Moving Average Convergence Divergence (MACD).
*   **Analyst Ratings:** The consensus rating is "Strong Buy".

**Bullish Predictions:**

*   **Upside Potential:** The average 12-month price target from analysts ranges from $294.40 to $297.03, suggesting a potential upside of over 18% from the current price. Some analysts have a high forecast of $340.
*   **Revenue Growth:** Amazon's revenue is expected to rise 12% in 2025.
*   **AWS and AI:** Growth is expected to be driven by AWS and Amazon's investments in AI.
*   **Long-Term Growth:** Long-term forecasts predict significant growth in the coming years.

**Bearish Predictions:**

*   **Potential Drop:** One source predicts the value of AMZN shares will drop by -0.70% and reach $242.71 per share by December 7, 2025.
*   **Sell Signal:** One analysis indicates that AMZN is flashing a massive 5-year sell signal.
*   **Competition:** Amazon faces competition in e-commerce.
*   **Regulation:** Regulatory concerns remain a risk.
*   **Horizontal Trend:** Given the current horizontal trend, one can expect Amazon stock with a 90% probability to be traded between $216.99 and $257.91 at the end of this 3-month period.


In [8]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

It appears you might be asking about two different companies:

*   **Metro-Goldwyn-Mayer (MGM):** This is the film and television studio. It was acquired by Amazon in 2022 and is now a subsidiary called Amazon MGM Studios. It does not appear to be traded publicly under its own ticker symbol anymore.
*   **MGM Resorts International:** This is a hospitality and entertainment company. Its stock ticker symbol is **MGM** and it is listed on the New York Stock Exchange (NYSE). As of November 7, 2025, its stock price was $32.66.


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Here's the requested data for AMZN (Amazon) stock:

**November 7, 2025:**

*   Open: $242.90
*   High: $244.90
*   Low: $238.49
*   Close: $244.41

**November 6, 2025:**

*   Open: $249.15
*   High: $250.38
*   Low: $242.17
*   Close: $243.04


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Okay, here's the AMZN (Amazon) stock data for the past month (October 8, 2025 - November 7, 2025). Note that I am unable to provide the exact data for each day, but I can provide representative values and trends based on available information. The data may not be 100% precise due to limitations in accessing specific historical intraday data.

| Date        | Open    | High    | Low     | Close   |
| ----------- | ------- | ------- | ------- | ------- |
| 2025-10-08  | ~$225   | ~$227   | ~$223   | ~$226   |
| 2025-10-15  | ~$230   | ~$232   | ~$228   | ~$231   |
| 2025-10-22  | ~$235   | ~$237   | ~$233   | ~$236   |
| 2025-10-29  | ~$240   | ~$242   | ~$238   | ~$241   |
| 2025-11-05  | ~$245   | ~$247   | ~$243   | ~$246   |
| 2025-11-06  | $249.15 | $250.38 | $242.17 | $243.04 |
| 2025-11-07  | $242.90 | $244.90 | $238.49 | $244.41 |

**General Trend:**

*   Over the past month, AMZN has generally shown an upward trend.
*   There have been fluctuations, but the overall direction has been positive.
*   The last few days have shown some volatility.

**Disclaimer:** This data is an approximation based on available information and general trends. For precise historical intraday data, you would typically need to consult a financial data provider or trading platform.
